In [1]:
import pandas as pd

In [2]:
train_data_save_path = "./extracted/SBIC.v2.trn.csv"
dev_data_save_path = "./extracted/SBIC.v2.dev.csv"
test_data_save_path = "./extracted/SBIC.v2.tst.csv"

In [3]:
from process_data import read_tgz_data

train_data = read_tgz_data(train_data_save_path)
dev_data = read_tgz_data(dev_data_save_path)
test_data = read_tgz_data(test_data_save_path)

Error occurred while extracting the file: [Errno 2] No such file or directory: '/Users/sabrina/Computational Social Science/final-project/SBIC.v2.tgz'
Error occurred while reading the file: [Errno 2] No such file or directory: '/Users/sabrina/Computational Social Science/final-project/extracted\\SBIC.v2.tst.csv'


In [5]:
import re

def get_context_str(category, target_group, target_stereotype):
    category, target_group, target_stereotype = str(category), str(target_group), str(target_stereotype)

    if target_stereotype.startswith(target_group):
        return " ".join((category, ":", target_stereotype))
    else:
        return " ".join((category, ":", target_group, target_stereotype))

def normalize_spacing(input):
    cleaned_string = input.strip()
    cleaned_string = ' '.join(cleaned_string.split())
    return cleaned_string

def remove_html_entities(text):
    return re.sub(r"&#[0-9]+;", "", text)

def remove_rt_username(text):
    return re.sub(r"RT @\w+\s*:", "", text)

def remove_beginning_and_ending_tags(text):
    text = re.sub(r"^(?:@[A-Za-z0-9_]+ )+", "", text)
    text = re.sub(r"(?: @[A-Za-z0-9_]+)+$", "", text)
    text = re.sub(r"^(?:@[A-Za-z0-9_]+[^\w\s]? )+", "", text)
    text = re.sub(r"^(?:\.\s*)?(?:@[A-Za-z0-9_]+ )+", "", text)
    return text

def remove_start_html(text):
    return re.sub(r"&gt;", "", text)

def remove_emojis(text):
    emoji_pattern = re.compile("["
                               "\U0001F600-\U0001F64F"
                               "\U0001F300-\U0001F5FF"
                               "\U0001F680-\U0001F6FF"
                               "\U0001F1E0-\U0001F1FF"
                               "]+", flags=re.UNICODE)
    return re.sub(emoji_pattern, "", text)

def clean_post(post):
    post = remove_html_entities(post)
    post = remove_rt_username(post)
    post = remove_beginning_and_ending_tags(post)
    post = remove_emojis(post)
    post = remove_start_html(post)
    post = normalize_spacing(post)
    post = re.sub(r"@", "", post)
    post = re.sub(r"#", "", post)
    return post

def merge_same_posts(posts):
    first_row = posts.iloc[0]
    posts = posts.fillna("nan")
    implications = posts["targetStereotype"].tolist()
    targeted_groups = posts["targetMinority"].tolist()
    targeted_categories = posts["targetCategory"].tolist()
    
    contexts = list(zip(targeted_categories, targeted_groups, implications))
    nan_context = ("nan", "nan", "nan")
    filtered_contexts = [context for context in contexts if context != nan_context]
    contexts = [get_context_str(tcat, tgroup, implication) for (tcat, tgroup, implication) in filtered_contexts]
    # print(f"Contexts: {contexts}")
    first_row["context"] = contexts

    raw_post = first_row["post"]
    first_row["post"] = clean_post(raw_post)
    return first_row



In [6]:
merged_train = train_data.groupby("post").apply(merge_same_posts).reset_index(drop = True)
merged_dev = dev_data.groupby("post").apply(merge_same_posts).reset_index(drop = True)
merged_test = test_data.groupby("post").apply(merge_same_posts).reset_index(drop = True)

C:\Users\sabri\AppData\Local\Temp\ipykernel_16520\602313721.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  merged_train = train_data.groupby("post").apply(merge_same_posts).reset_index(drop = True)
C:\Users\sabri\AppData\Local\Temp\ipykernel_16520\602313721.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  merged_dev = dev_data.groupby("post").apply(merge_same_posts).reset_index(drop = True)
C:\Users

In [8]:
# Save merged
merged_train_savepath = "./data/merged/merged_train.csv"
merged_dev_savepath = "./data/merged/merged_dev.csv"
merged_test_savepath = "./data/merged/merged_test.csv"

merged_train.to_csv(merged_train_savepath, index=False)
merged_dev.to_csv(merged_dev_savepath, index=False)
merged_test.to_csv(merged_test_savepath, index=False)

In [9]:
def safe_len(x):
    return len(x) if isinstance(x, list) else 0

In [10]:
# Save with impl
merged_train_impl_savepath = "./data/with_impl/merged_train.csv"
merged_dev_impl_savepath = "./data/with_impl/merged_dev.csv"
merged_test_impl_savepath = "./data/with_impl/merged_test.csv"

merged_train_impl = merged_train[merged_train["context"].map(safe_len) > 0]
merged_dev_impl = merged_dev[merged_dev["context"].map(safe_len) > 0]
merged_test_impl = merged_test[merged_test["context"].map(safe_len) > 0]

merged_train_impl.to_csv(merged_train_impl_savepath, index=False)
merged_dev_impl.to_csv(merged_dev_impl_savepath, index=False)
merged_test_impl.to_csv(merged_test_impl_savepath, index=False)

In [11]:
# Testing to see tokenizer outputs
import torch
from transformers import FunnelTokenizer

tokenizer = FunnelTokenizer.from_pretrained("funnel-transformer/small")

# Tests
texts = ["Hello, world!", "Here is a test string!"]

tokenized_output = tokenizer.batch_encode_plus(texts, padding=True, truncation=True, return_tensors="pt")
print(tokenized_output)

d:\autoencoding-social-bias\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'input_ids': tensor([[ 101, 7592, 1010, 2088,  999,  102,    0,    0],
        [ 101, 2182, 2003, 1037, 3231, 5164,  999,  102]]), 'token_type_ids': tensor([[2, 0, 0, 0, 0, 0, 0, 0],
        [2, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1]])}


d:\autoencoding-social-bias\.venv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [12]:
import torch
from transformers import FunnelTokenizer, FunnelModel

tokenizer = FunnelTokenizer.from_pretrained("funnel-transformer/small")
model = FunnelModel.from_pretrained("funnel-transformer/small")

d:\autoencoding-social-bias\.venv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [13]:
import numpy as np
from transformers import FunnelTokenizer, FunnelModel

tokenizer = FunnelTokenizer.from_pretrained("funnel-transformer/small")
MAX_TOK_LEN = 70
# Tokenize post helper function
def tokenize_post(row):
    context_str = " <sep> ".join(row["context"]) + " <sep>"
    post_str = "<cls> " + row["post"]
    input_str = post_str + context_str
    
    post_data = row["post"]
    # print(f"Input str: {input_str}")
    tokenized_output = tokenizer(input_str, padding="max_length", add_special_tokens=False, truncation=True, max_length = MAX_TOK_LEN, return_tensors="pt")
    
    input_ids = tokenized_output["input_ids"].numpy()
    attention_mask = tokenized_output["attention_mask"].numpy()
    token_type_ids = tokenized_output["token_type_ids"].numpy()
    
    tokenized_post = tokenizer(post_data, padding="max_length", max_length = MAX_TOK_LEN, truncation=True, return_tensors="pt")
    target_labels = tokenized_post["input_ids"].numpy()
    # token_type_ids_np = np.zeros_like(tokenized_output["token_type_ids"])
    
    return {
        "input_ids": input_ids,
        "token_type_ids": token_type_ids,
        "attention_mask": attention_mask,
        "target_labels": target_labels
    }

def tokenize_post_no_context(row):
    post_str = "<cls> " + row["post"] + " <sep>"
    input_str = post_str
    
    post_data = row["post"]
    # print(f"Input str: {input_str}")
    tokenized_output = tokenizer(input_str, padding="max_length", add_special_tokens=False, truncation=True, max_length = MAX_TOK_LEN, return_tensors="pt")
    
    input_ids = tokenized_output["input_ids"].numpy()
    attention_mask = tokenized_output["attention_mask"].numpy()
    token_type_ids = tokenized_output["token_type_ids"].numpy()
    
    tokenized_post = tokenizer(post_data, padding="max_length", max_length = MAX_TOK_LEN, truncation=True, return_tensors="pt")
    target_labels = tokenized_post["input_ids"].numpy()
    # token_type_ids_np = np.zeros_like(tokenized_output["token_type_ids"])
    
    return {
        "input_ids": input_ids,
        "token_type_ids": token_type_ids,
        "attention_mask": attention_mask,
        "target_labels": target_labels
    }

def t5_tokenize_post_reconstruction_prompt(row):
    summary = row["summary"]
    context = row["context"]
    input_prompt = f"Based on this summary: <{summary}>, and the implications of the post <{context}>, reconstruct the post."
    tokenized_output = tokenizer.encode(input_prompt, return_tensors="pt")
    
    return tokenized_output.numpy()

# Contexts is a list of strings
# def tokenize_context(contexts):
#     input_ids = []
#     attention_masks = []
#     for context in contexts:
#         tokenized_output = tokenizer(context, padding=True, truncation=True, return_tensors="pt")
#         context_tokens.append((tokenized_output["input_ids"], tokenized_output["token_type_ids"], tokenized_output["attention_mask"]))
    

In [32]:
# ## FOR FUNNEL TRANSFORMER ###

# # Tokenize and add to df
# train_tokens = merged_train.apply(tokenize_post, axis = 1)
# train_tokens_df = pd.DataFrame(train_tokens.tolist(), columns=["input_ids", "token_type_ids", "attention_mask", "target_labels"])
# train_tokenized_df = pd.concat([merged_train.reset_index(drop=True), train_tokens_df.reset_index(drop=True)], axis=1)

# print(train_tokenized_df.columns.tolist())

# # Tokenize and add to df
# dev_tokens = merged_dev.apply(tokenize_post, axis = 1)
# dev_tokens_df = pd.DataFrame(dev_tokens.tolist(), columns=["input_ids", "token_type_ids", "attention_mask", "target_labels"])
# dev_tokenized_df = pd.concat([merged_dev.reset_index(drop=True), dev_tokens_df.reset_index(drop=True)], axis=1)


# # Tokenize and add to df
# test_tokens = merged_test.apply(tokenize_post, axis = 1)
# test_tokens_df = pd.DataFrame(test_tokens.tolist(), columns=["input_ids", "token_type_ids", "attention_mask", "target_labels"])
# test_tokenized_df = pd.concat([merged_test.reset_index(drop=True), test_tokens_df.reset_index(drop=True)], axis=1)


# train_tokenized_df.to_json("./data/tokenized/train_tokenized.json", orient = "records", lines = True)
# dev_tokenized_df.to_json("./data/tokenized/dev_tokenized.json", orient = "records", lines = True)
# test_tokenized_df.to_json("./data/tokenized/test_tokenized.json", orient = "records", lines = True)

['whoTarget', 'intentYN', 'sexYN', 'sexReason', 'offensiveYN', 'annotatorGender', 'annotatorMinority', 'sexPhrase', 'speakerMinorityYN', 'WorkerId', 'HITId', 'annotatorPolitics', 'annotatorRace', 'annotatorAge', 'post', 'targetMinority', 'targetCategory', 'targetStereotype', 'dataSource', 'context', 'input_ids', 'token_type_ids', 'attention_mask', 'target_labels']


In [14]:
## FOR FUNNEL TRANSFORMER ### ### FOR WITH_IMPL

# Tokenize and add to df
train_tokens = merged_train_impl.apply(tokenize_post, axis = 1)
train_tokens_df = pd.DataFrame(train_tokens.tolist(), columns=["input_ids", "token_type_ids", "attention_mask", "target_labels"])
train_tokenized_df = pd.concat([merged_train_impl.reset_index(drop=True), train_tokens_df.reset_index(drop=True)], axis=1)

print(train_tokenized_df.columns.tolist())

# Tokenize and add to df
dev_tokens = merged_dev_impl.apply(tokenize_post, axis = 1)
dev_tokens_df = pd.DataFrame(dev_tokens.tolist(), columns=["input_ids", "token_type_ids", "attention_mask", "target_labels"])
dev_tokenized_df = pd.concat([merged_dev_impl.reset_index(drop=True), dev_tokens_df.reset_index(drop=True)], axis=1)


# Tokenize and add to df
test_tokens = merged_test_impl.apply(tokenize_post, axis = 1)
test_tokens_df = pd.DataFrame(test_tokens.tolist(), columns=["input_ids", "token_type_ids", "attention_mask", "target_labels"])
test_tokenized_df = pd.concat([merged_test_impl.reset_index(drop=True), test_tokens_df.reset_index(drop=True)], axis=1)


train_tokenized_df.to_json("./data/tokenized/train_tokenized.json", orient = "records", lines = True)
dev_tokenized_df.to_json("./data/tokenized/dev_tokenized.json", orient = "records", lines = True)
test_tokenized_df.to_json("./data/tokenized/test_tokenized.json", orient = "records", lines = True)

['whoTarget', 'intentYN', 'sexYN', 'sexReason', 'offensiveYN', 'annotatorGender', 'annotatorMinority', 'sexPhrase', 'speakerMinorityYN', 'WorkerId', 'HITId', 'annotatorPolitics', 'annotatorRace', 'annotatorAge', 'post', 'targetMinority', 'targetCategory', 'targetStereotype', 'dataSource', 'context', 'input_ids', 'token_type_ids', 'attention_mask', 'target_labels']


In [99]:
import ast
def convert_string_to_list(string):
	return ast.literal_eval(string)

In [44]:
import pandas as pd
# For loading data back in rather than rerunning above
merged_train_savepath = "./data/merged/merged_train.csv"
merged_dev_savepath = "./data/merged/merged_dev.csv"
merged_test_savepath = "./data/merged/merged_test.csv"

merged_train = pd.read_csv(merged_train_savepath)
merged_dev = pd.read_csv(merged_dev_savepath)
merged_test = pd.read_csv(merged_test_savepath)

merged_train_impl_savepath = "./data/with_impl/merged_train.csv"
merged_dev_impl_savepath = "./data/with_impl/merged_dev.csv"
merged_test_impl_savepath = "./data/with_impl/merged_test.csv"

merged_train_impl = pd.read_csv(merged_train_impl_savepath)
merged_dev_impl = pd.read_csv(merged_dev_impl_savepath)
merged_test_impl = pd.read_csv(merged_test_impl_savepath)

train_tokenized_df = pd.read_json("./data/tokenized/train_tokenized.json", orient = "records", lines = True)
dev_tokenized_df = pd.read_json("./data/tokenized/dev_tokenized.json", orient = "records", lines = True)
test_tokenized_df = pd.read_json("./data/tokenized/test_tokenized.json", orient = "records", lines = True)


In [45]:
# Dataloader
from torch.utils.data import Dataset, DataLoader

class BiasDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
        
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        # print("test")
        input_ids = self.dataset.loc[idx, "input_ids"]
        # print(f"Input ids: {input_ids}")
        attention_mask = self.dataset.loc[idx, "attention_mask"]
        token_type_ids = self.dataset.loc[idx, "token_type_ids"]
        target_labels = self.dataset.loc[idx, "target_labels"]
        
        # print(type(input_ids))
        # print(type(attention_mask))
        # print(type(token_type_ids))
        # print(type(target_labels))
        # print("test2")
        # Create input tensors
        inputs = {
            "input_ids": torch.tensor(input_ids, dtype=torch.long).squeeze(dim=0),
            "attention_mask": torch.tensor(attention_mask, dtype=torch.long).squeeze(dim=0),
            "token_type_ids": torch.tensor(token_type_ids, dtype=torch.long).squeeze(dim=0),
            "target_labels": torch.tensor(target_labels, dtype=torch.long).squeeze(dim=0)
        }

        # print("test3")
        
        return inputs

In [46]:
train_dataloader = DataLoader(
    BiasDataset(train_tokenized_df),
    batch_size=32,
    shuffle=True
)

test_dataloader = DataLoader(
    BiasDataset(test_tokenized_df.iloc[:50]),
    batch_size=1,
    shuffle=False
)

In [47]:
import torch
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print(f"Using {device} device")

Using cuda device


In [48]:
import torch.nn as nn

class RegenerativeTransformer(nn.Module):
    def __init__(self, encoder, decoder):
        super(RegenerativeTransformer, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.embedding = nn.Embedding(30522, 768)
        self.output = nn.Linear(768, 30522)

    def forward(self, src, tgt, src_mask=None, tgt_mask=None):
        memory = self.encoder(src, attention_mask=src_mask)[0]  # Ensure output matches expected format
        output = self.decoder(tgt, memory, tgt_mask=tgt_mask, memory_mask=src_mask)
        return output

/Users/sabrina/Computational Social Science/final-project/local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [10]:
# import tqdm
# import torch.nn.functional as F

# MAX_LENGTH = 50
# def train_loop(dataloader, model, loss_fn, encoder_optimizer, decoder_optimizer, start_token_id):
#     train_loss = 0
#     # set the model to training model
#     model.train()
#     iter_count = 0
#     # for batch in dataloader:
#     for batch in tqdm.tqdm(dataloader):
#         encoder_optimizer.zero_grad()
#         decoder_optimizer.zero_grad()
        
#         # previous tokens
#         input_ids = batch["input_ids"].to(device)
#         attention_mask = batch["attention_mask"].to(device)
#         token_type_ids = batch["token_type_ids"].to(device)
#         target_labels = batch["target_labels"][:, :MAX_LENGTH].to(device)
        
#         target_pad = MAX_LENGTH - target_labels.size(1)
#         if target_pad > 0:
#              target_labels = F.pad(target_labels, (0, target_pad), "constant", 0)
        
#         # print(f"Input ids: {input_ids.shape}")
#         # print(f"Attention_mask: {attention_mask.shape}")
#         encoder_outputs = model.encoder(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        
#         encoder_state = encoder_outputs.last_hidden_state
#         # Decoder part initiated
        
#         start_token_id = torch.tensor([start_token_id], dtype=torch.long, device=device)
#         start_token_embed = model.embedding(start_token_id)
#         decoder_input = start_token_embed.repeat(input_ids.size(0), 1, 1)
        
#         # decoder_input = torch.tensor([model.embedding(start_token_id)]*input_ids.size(0), device=device)
#         outputs = []

#         for i in range(MAX_LENGTH):
#             # print(f"Decoder input: {decoder_input.shape}")
#             # print(f"Encoder output: {encoder_state.shape}")
#             decoder_output = model.decoder(decoder_input, encoder_state)
            
#             logits = model.output(decoder_output.squeeze(dim = 1))
#             outputs.append(logits)
            
#             decoder_input = target_labels[:, i]
#             decoder_input = model.embedding(decoder_input).unsqueeze(dim = 1)
            

#             # _, topi = logits.topk(1)
#             # decoder_input = topi.squeeze().detach()
#             # decoder_input = model.embedding(decoder_input).repeat(input_ids.size(0), 1, 1)

#         outputs = torch.stack(outputs, dim=1)  # [batch_size, MAX_LENGTH, vocab_size]
#         # print(f"OUTPUTS: {outputs.shape}")
#         # print(f"TARGET LABELS: {target_labels.shape}")
        
#         # preds = torch.argmax(outputs, dim = 2)
#         mask = target_labels != 0
#         # loss = loss_fn(outputs.view(-1, outputs.size(-1)), target_labels.view(-1))
#         # # print(f"Loss shape: {loss.shape}")
#         # loss = loss.view(target_labels.shape)
#         # masked_loss = loss * mask.float()

#         # loss_sum = masked_loss.sum()
#         # num_valid_tokens = mask.sum()
#         # loss = loss_sum / num_valid_tokens.float()

#         loss = loss_fn(outputs.view(-1, outputs.size(-1)), target_labels.view(-1))
#         mask = target_labels != 0
#         masked_loss = loss * mask.view(-1).float()
#         loss = masked_loss.sum() / mask.sum()
#         train_loss += loss.item()


#         if iter_count % 5 == 0:
#             print(f"Loss: {loss.item()}")
#         loss.backward()

#         encoder_optimizer.step()
#         decoder_optimizer.step()

#         iter_count += 1
    
#     return train_loss / len(dataloader)

# def test_loop(dataloader, model, loss_fn, tokenizer, start_token_id):

#     all_sentences = []
#     model.eval()
#     val_loss = 0
#     with torch.no_grad():
#         for batch in tqdm.tqdm(dataloader):
#             # previous tokens
#             input_ids = batch["input_ids"].to(device)
#             attention_mask = batch["attention_mask"].to(device)
#             token_type_ids = batch["token_type_ids"].to(device)
#             target_labels = batch["target_labels"][:, :MAX_LENGTH].to(device)
            
#             target_pad = MAX_LENGTH - target_labels.size(1)
#             if target_pad > 0:
#                 target_labels = F.pad(target_labels, (0, target_pad), "constant", 0)
            
#             # print(f"Input ids: {input_ids.shape}")
#             # print(f"Attention_mask: {attention_mask.shape}")
#             encoder_outputs = model.encoder(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
            
#             encoder_state = encoder_outputs.last_hidden_state
            
#             start_token_id = torch.tensor([start_token_id], dtype=torch.long, device=device)
#             start_token_embed = model.embedding(start_token_id)
#             decoder_input = start_token_embed.repeat(input_ids.size(0), 1, 1)
            
#             # decoder_input = torch.tensor([model.embedding(start_token_id)]*input_ids.size(0), device=device)  # Start token
#             outputs = []

#             for i in range(MAX_LENGTH):
#                 print(f"Decoder input: {decoder_input.shape}")
#                 # print(f"Encoder output: {encoder_state.shape}")
#                 decoder_output = model.decoder(decoder_input, encoder_state)
                
#                 logits = model.output(decoder_output.squeeze(dim = 1))
#                 outputs.append(logits)

#                 _, topi = logits.topk(1)
#                 decoder_input = topi.squeeze().detach()
#                 decoder_input = model.embedding(decoder_input).unsqueeze(dim = 0).unsqueeze(dim = 0)

#             outputs = torch.stack(outputs, dim=1)  # [batch_size, MAX_LENGTH, vocab_size]
#             # print(f"OUTPUTS: {outputs.shape}")
#             # print(f"TARGET LABELS: {target_labels.shape}")
            
#             preds = torch.argmax(outputs, dim = 2)
#             # mask = target_labels != 0
#             # loss = loss_fn(outputs.view(-1, outputs.size(-1)), target_labels.view(-1))
#             # loss = loss.view(target_labels.shape)
#             # masked_loss = loss * mask.float()

#             loss = loss_fn(outputs.view(-1, outputs.size(-1)), target_labels.view(-1))
#             mask = target_labels != 0
#             masked_loss = loss * mask.view(-1).float()
#             loss = masked_loss.sum() / mask.sum()

#             loss_sum = masked_loss.sum()
#             num_valid_tokens = mask.sum()
#             loss = loss_sum / num_valid_tokens.float()
#             print(f"Loss: {loss.item()}")
#             val_loss += loss.item()
            
#             decoded_sentences = [tokenizer.decode(pred, skip_special_tokens=True) for pred in preds]
#             all_sentences.extend(decoded_sentences)

#     return val_loss / len(dataloader), all_sentences
    
        
    

In [63]:
import tqdm
import torch.nn.functional as F
import random

MAX_LENGTH = 50
def train_loop(dataloader, model, loss_fn, encoder_optimizer, decoder_optimizer, start_token_id):
    train_loss = 0
    # set the model to training model
    model.train()
    iter_count = 0
    # for batch in dataloader:
    for batch in tqdm.tqdm(dataloader):
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        
        # previous tokens
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        token_type_ids = batch["token_type_ids"].to(device)
        target_labels = batch["target_labels"][:, :MAX_LENGTH].to(device)
        
        target_pad = MAX_LENGTH - target_labels.size(1)
        if target_pad > 0:
             target_labels = F.pad(target_labels, (0, target_pad), "constant", 0)
        
        # print(f"Input ids: {input_ids.shape}")
        # print(f"Attention_mask: {attention_mask.shape}")
        encoder_outputs = model.encoder(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        
        encoder_state = encoder_outputs.last_hidden_state
        # Decoder part initiated
        
        start_token_id = torch.tensor([start_token_id], dtype=torch.long, device=device)
        start_token_embed = model.embedding(start_token_id)
        decoder_input = start_token_embed.repeat(input_ids.size(0), 1, 1)
        # print(f"Decoder input: {decoder_input.shape}")
        # decoder_input = torch.tensor([model.embedding(start_token_id)]*input_ids.size(0), device=device)
        outputs = []

        teacher_forcing_ratio = 0.5

        for i in range(MAX_LENGTH):
            # print(f"Decoder input: {decoder_input.shape}")
            # print(f"Encoder output: {encoder_state.shape}")
            decoder_output = model.decoder(decoder_input, encoder_state)
            
            logits = model.output(decoder_output)[:, -1, :]
            outputs.append(logits)
            # print(f"Logits: {logits.shape}")

            _, topi = logits.topk(1)
            
            use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
            if use_teacher_forcing and i < target_labels.size(1) - 1:
                next_token = target_labels[:, i + 1]
                # print(f"Teacher forced: {next_token.shape}")
            else:
                next_token = topi.squeeze().detach()
                # print(f"Autoregresesive: {next_token.shape}")

            # print("next tok")
            # print(next_token.shape)
            # print(f"I: {i}")
            next_input = model.embedding(next_token).unsqueeze(dim = 1)
            # print(f"Next input: {next_input.shape}")
            
            decoder_input = torch.cat((decoder_input, next_input), dim=1)
            # print(f"Decoder input: {decoder_input.shape}")

            # _, topi = logits.topk(1)
            # decoder_input = topi.squeeze().detach()
            # decoder_input = model.embedding(decoder_input).repeat(input_ids.size(0), 1, 1)

        outputs = torch.stack(outputs, dim=1)  # [batch_size, MAX_LENGTH, vocab_size]
        # print(f"OUTPUTS: {outputs.shape}")
        # print(f"TARGET LABELS: {target_labels.shape}")
        
        # preds = torch.argmax(outputs, dim = 2)
        mask = target_labels != 0
        # loss = loss_fn(outputs.view(-1, outputs.size(-1)), target_labels.view(-1))
        # # print(f"Loss shape: {loss.shape}")
        # loss = loss.view(target_labels.shape)
        # masked_loss = loss * mask.float()

        # loss_sum = masked_loss.sum()
        # num_valid_tokens = mask.sum()
        # loss = loss_sum / num_valid_tokens.float()

        loss = loss_fn(outputs.view(-1, outputs.size(-1)), target_labels.view(-1))
        mask = target_labels != 0
        masked_loss = loss * mask.view(-1).float()
        loss = masked_loss.sum() / mask.sum()
        train_loss += loss.item()


        if iter_count % 5 == 0:
            print(f"Loss: {loss.item()}")
        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

        iter_count += 1
    
    return train_loss / len(dataloader)

def test_loop(dataloader, model, loss_fn, tokenizer, start_token_id):

    all_sentences = []
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in tqdm.tqdm(dataloader):
            # previous tokens
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            token_type_ids = batch["token_type_ids"].to(device)
            target_labels = batch["target_labels"][:, :MAX_LENGTH].to(device)
            
            target_pad = MAX_LENGTH - target_labels.size(1)
            if target_pad > 0:
                target_labels = F.pad(target_labels, (0, target_pad), "constant", 0)
            
            # print(f"Input ids: {input_ids.shape}")
            # print(f"Attention_mask: {attention_mask.shape}")
            encoder_outputs = model.encoder(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
            
            encoder_state = encoder_outputs.last_hidden_state
            
            start_token_id = torch.tensor([start_token_id], dtype=torch.long, device=device)
            start_token_embed = model.embedding(start_token_id)
            decoder_input = start_token_embed.repeat(input_ids.size(0), 1, 1)
            
            # decoder_input = torch.tensor([model.embedding(start_token_id)]*input_ids.size(0), device=device)  # Start token
            outputs = []

            for i in range(MAX_LENGTH):
                # print(f"Decoder input: {decoder_input.shape}")
                # print(f"Encoder output: {encoder_state.shape}")
                decoder_output = model.decoder(decoder_input, encoder_state)
               
                logits = model.output(decoder_output)[:, -1, :]
                
                outputs.append(logits)

                _, topi = logits.topk(1)
                next_token = topi.squeeze().detach()
                # print(f"First next token: {next_token.shape}")
                next_token = model.embedding(next_token).unsqueeze(dim = 0).unsqueeze(dim = 0)
                # print(f"Next token: {next_token.shape}")
                # print(f"Decoder input {decoder_input.shape}")
                decoder_input = torch.cat((decoder_input, next_token), dim = 1)

            outputs = torch.stack(outputs, dim=1)  # [batch_size, MAX_LENGTH, vocab_size]
            # print(f"OUTPUTS: {outputs.shape}")
            # print(f"TARGET LABELS: {target_labels.shape}")
            
            preds = torch.argmax(outputs, dim = 2)
            # mask = target_labels != 0
            # loss = loss_fn(outputs.view(-1, outputs.size(-1)), target_labels.view(-1))
            # loss = loss.view(target_labels.shape)
            # masked_loss = loss * mask.float()

            loss = loss_fn(outputs.view(-1, outputs.size(-1)), target_labels.view(-1))
            mask = target_labels != 0
            masked_loss = loss * mask.view(-1).float()
            loss = masked_loss.sum() / mask.sum()

            loss_sum = masked_loss.sum()
            num_valid_tokens = mask.sum()
            loss = loss_sum / num_valid_tokens.float()
            print(f"Loss: {loss.item()}")
            val_loss += loss.item()
           
            decoded_sentences = [tokenizer.decode(pred, skip_special_tokens=True) for pred in preds]
            all_sentences.extend(decoded_sentences)

    return val_loss / len(dataloader), all_sentences
    
        
    

In [50]:
import torch
from torch.nn import Transformer, TransformerDecoder, TransformerDecoderLayer
from transformers import FunnelTokenizer, FunnelModel
import torch.optim as optim

encoder = FunnelModel.from_pretrained("funnel-transformer/small")
encoder_tokenizer = FunnelTokenizer.from_pretrained("funnel-transformer/small")
print(encoder_tokenizer.vocab_size)
decoder_layer = TransformerDecoderLayer(d_model=768, nhead=8, batch_first = True)
decoder = TransformerDecoder(decoder_layer, num_layers = 6)

regenerative_model = RegenerativeTransformer(encoder, decoder).to(device)

encoder_optimizer = optim.Adam(regenerative_model.encoder.parameters(), lr=0.001)
decoder_optimizer = optim.Adam(regenerative_model.decoder.parameters(), lr=0.01)

loss_fn = nn.CrossEntropyLoss(reduction='none')

d:\autoencoding-social-bias\.venv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


30522


In [51]:
print(encoder_tokenizer.pad_token)
print(encoder_tokenizer.convert_tokens_to_ids(encoder_tokenizer.pad_token))
print("test")

<pad>
0
test


In [52]:
bos_token_id = encoder_tokenizer.convert_tokens_to_ids(encoder_tokenizer.bos_token)
# train_loop(train_dataloader, regenerative_model, loss_fn, encoder_optimizer, decoder_optimizer, bos_token_id)

In [64]:
test_loop(test_dataloader, regenerative_model, loss_fn, encoder_tokenizer, bos_token_id)

  0%|          | 0/50 [00:00<?, ?it/s]

Decoder input: torch.Size([1, 1, 768])
First next token: torch.Size([])
Next token: torch.Size([1, 1, 768])
Decoder input torch.Size([1, 1, 768])
Decoder input: torch.Size([1, 2, 768])
First next token: torch.Size([])
Next token: torch.Size([1, 1, 768])
Decoder input torch.Size([1, 2, 768])
Decoder input: torch.Size([1, 3, 768])
First next token: torch.Size([])
Next token: torch.Size([1, 1, 768])
Decoder input torch.Size([1, 3, 768])
Decoder input: torch.Size([1, 4, 768])
First next token: torch.Size([])
Next token: torch.Size([1, 1, 768])
Decoder input torch.Size([1, 4, 768])
Decoder input: torch.Size([1, 5, 768])
First next token: torch.Size([])
Next token: torch.Size([1, 1, 768])
Decoder input torch.Size([1, 5, 768])
Decoder input: torch.Size([1, 6, 768])
First next token: torch.Size([])
Next token: torch.Size([1, 1, 768])
Decoder input torch.Size([1, 6, 768])
Decoder input: torch.Size([1, 7, 768])
First next token: torch.Size([])
Next token: torch.Size([1, 1, 768])
Decoder input to

  0%|          | 0/50 [00:21<?, ?it/s]


KeyboardInterrupt: 

In [62]:
torch.cuda.empty_cache()
EPOCHS = 5

train_losses = []
val_losses = []
for epoch in range(EPOCHS):
	train_loss = train_loop(train_dataloader, regenerative_model, loss_fn, encoder_optimizer, decoder_optimizer, bos_token_id)
	print(f"Epoch: {train_loss}")
	train_losses.append(train_loss)
	if epoch % 1 == 0:
		val_loss, val_sentences = test_loop(test_dataloader, regenerative_model, loss_fn, encoder_tokenizer, bos_token_id)
		val_losses.append(val_loss)
		checkpoint = {
			"model": regenerative_model.state_dict(),
			"encoder_optimizer": encoder_optimizer.state_dict(),
			"decoder_optimizer": decoder_optimizer.state_dict(),
			"train_losses": train_losses,
			"val_losses": val_losses,
			"val_sentences": val_sentences,
			"epoch": epoch
		}
		torch.save(checkpoint, f"./checkpoints/checkpoint_{epoch}.pt")


print(f"Train Losses: {train_losses}")
print(f"Val Losses: {val_losses}")

for sent in val_sentences:
	print(sent)
	

  0%|          | 0/380 [00:00<?, ?it/s]

Decoder input: torch.Size([32, 1, 768])
Logits: torch.Size([32, 30522])
Teacher forced: torch.Size([32])
next tok
torch.Size([32])
I: 0
Next input: torch.Size([32, 1, 768])
Decoder input: torch.Size([32, 2, 768])
Logits: torch.Size([32, 30522])
Teacher forced: torch.Size([32])
next tok
torch.Size([32])
I: 1
Next input: torch.Size([32, 1, 768])
Decoder input: torch.Size([32, 3, 768])
Logits: torch.Size([32, 30522])
Teacher forced: torch.Size([32])
next tok
torch.Size([32])
I: 2
Next input: torch.Size([32, 1, 768])
Decoder input: torch.Size([32, 4, 768])
Logits: torch.Size([32, 30522])
Autoregresesive: torch.Size([32])
next tok
torch.Size([32])
I: 3
Next input: torch.Size([32, 1, 768])
Decoder input: torch.Size([32, 5, 768])
Logits: torch.Size([32, 30522])
Autoregresesive: torch.Size([32])
next tok
torch.Size([32])
I: 4
Next input: torch.Size([32, 1, 768])
Decoder input: torch.Size([32, 6, 768])
Logits: torch.Size([32, 30522])
Autoregresesive: torch.Size([32])
next tok
torch.Size([32])
I

  0%|          | 0/380 [00:23<?, ?it/s]


KeyboardInterrupt: 

In [97]:
from transformers import FunnelTokenizer, FunnelModel
tokenizer = FunnelTokenizer.from_pretrained("funnel-transformer/small")
model = FunnelModel.from_pretrained("funnel-transformer/small")
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
print(output)
print(output.last_hidden_state.shape)

/Users/sabrina/Computational Social Science/final-project/local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


BaseModelOutput(last_hidden_state=tensor([[[ 0.1376, -0.3090,  0.6842,  ..., -0.4041,  0.8737, -0.7017],
         [-0.1957, -0.2551,  0.0395,  ...,  0.2188,  0.1387,  0.1632],
         [-0.2267,  0.3701,  0.0961,  ...,  0.0500, -0.0244, -0.1002],
         ...,
         [-0.5314,  0.7220,  0.2493,  ...,  0.0725,  0.0889, -0.0084],
         [-0.1779,  0.1923,  0.5646,  ...,  0.4919,  0.7747, -1.1734],
         [-0.1268,  0.0407,  0.0438,  ...,  0.6065,  1.0749, -1.3630]]],
       grad_fn=<NativeLayerNormBackward0>), hidden_states=None, attentions=None)
torch.Size([1, 12, 768])
